In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
from tqdm import tqdm

In [58]:
def non_shuffling_train_test_split(X, y, test_size=0.2):
    i = int((1 - test_size) * X.shape[0]) + 1
    X_train, X_test = np.split(X, [i])
    y_train, y_test = np.split(y, [i])
    return X_train, X_test, y_train, y_test

In [76]:
def compress_rows(df_seq):
    data = {}
    for game in df_seq.index.unique():
        data[game] = []
    for game in tqdm(df_seq.index.unique()):
        data[game].append(df_seq[df_seq.columns].loc[game].to_numpy())
    return data

In [80]:
df = pd.read_csv("../Data/single_dataset.csv", low_memory=False)
df.dropna(axis=0, how='any', inplace=True)

#enc characters
enc_p1 = LabelEncoder()
enc_p2 = LabelEncoder()

enc_p1.fit(df['CHAR_P1'])
enc_p2.fit(df['CHAR_P2'])
df['CHAR_P1'] = enc_p1.transform(df['CHAR_P1'])
df['CHAR_P2'] = enc_p2.transform(df['CHAR_P2'])

X_cols = ['CHAR_P1', 'CHAR_P2', 'S_airborne_P1',
   'S_damage_P1', 'S_direction_P1', 'S_hit_stun_P1', 'S_position_x_P1',
   'S_position_y_P1', 'S_shield_P1', 'S_state_P1', 'S_state_age_P1',
   'S_stocks_P1', 'S_airborne_P2', 'S_damage_P2', 'S_direction_P2',
   'S_hit_stun_P2', 'S_position_x_P2', 'S_position_y_P2', 'S_shield_P2',
   'S_state_P2', 'S_state_age_P2', 'S_stocks_P2']

y_cols = ['B_damage_P1',
   'B_direction_P1', 'B_joystick_x_P1', 'B_joystick_y_P1',
   'B_position_x_P1', 'B_position_y_P1', 'B_cstick_x_P1', 'B_cstick_y_P1',
   'B_state_P1', 'B_raw_analog_P1', 'B_buttons_physical_P1',
   'B_triggers_physical_l_P1', 'B_triggers_physical_r_P1',
   'B_triggers_logical_P1']

df['S_airborne_P1'] = df['S_airborne_P1'].astype(bool).astype(float)
df['S_airborne_P2'] = df['S_airborne_P2'].astype(bool).astype(float)

df_seq = df.set_index(df['Game_ID'])
df_seq.drop('Game_ID', axis = 1, inplace=True)

X = df_seq[X_cols]
y = df_seq[y_cols]

In [81]:
X_train, X_test, y_train, y_test = non_shuffling_train_test_split(X, y)

In [66]:
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)


y_train = y_scaler.transform(y_train)
y_test = y_scaler.transform(y_test)

In [84]:
datasets = [X_train, X_test, y_train, y_test]

datasets_seq = []

for d in datasets:
    datasets_seq.append(compress_rows(d))

100%|██████████| 4/4 [00:00<00:00, 363.58it/s]


In [87]:
class FrameDataset(Dataset):
    def __init__(self,X,y):
        self.x_train=torch.tensor(X,dtype=torch.float32)
        self.y_train=torch.tensor(y,dtype=torch.float32)
        
    def __len__(self):
        return len(self.y_train)
  
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]

In [88]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [89]:
input_size = dataset.x_train.shape[1]
output_size = dataset.y_train.shape[1]
hidden_size = 128

epochs = 10

model = RNN(input_size, n_hidden, n_categories)

learning_rate = 0.1

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
criterion = nn.MSELoss()

SyntaxError: invalid syntax (Temp/ipykernel_2316/1346302020.py, line 7)

In [ ]:
iter = 0
for epoch in range(epochs):
    train_losses = []
    for i, (data, labels) in enumerate(train_loader):

        optimizer.zero_grad()
        
        outputs = model(data)
        loss = criterion(outputs, labels)
        
        loss.backward()
        train_losses.append(loss.item())
        
        optimizer.step()
      
    test_losses = []
    
    for data, labels in test_loader:
        outputs = model(data)
        loss = criterion(outputs, labels)
        test_losses.append(loss.item())
        
    print('epoch {}, train loss {}'.format(epoch, np.average(train_losses)))
    print('epoch {}, test loss {}'.format(epoch, np.average(test_losses)))
    print(' ')
        